In [1]:
%%time
import warnings
warnings.filterwarnings("ignore")
import pickle
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import multiprocessing
from annoy import AnnoyIndex
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer


CPU times: total: 1.77 s
Wall time: 2.2 s


<h2>Load Data</h2>

In [2]:
%%time
df = pd.read_csv('NEET_2024_RESULTS.csv')
df.dropna(inplace=True)

CPU times: total: 1.72 s
Wall time: 1.86 s


In [3]:
df.head()

,dummy_srlno,marks,state,city,center_name,center_number
0,1101010001,46,ANDAMAN AND NICOBAR ISLANDS (UT),PORT BLAIR,KENDRIYA VIDYALAYA NO. 1,110101
1,1101010002,226,ANDAMAN AND NICOBAR ISLANDS (UT),PORT BLAIR,KENDRIYA VIDYALAYA NO. 1,110101
2,1101010003,449,ANDAMAN AND NICOBAR ISLANDS (UT),PORT BLAIR,KENDRIYA VIDYALAYA NO. 1,110101
3,1101010004,424,ANDAMAN AND NICOBAR ISLANDS (UT),PORT BLAIR,KENDRIYA VIDYALAYA NO. 1,110101
4,1101010005,222,ANDAMAN AND NICOBAR ISLANDS (UT),PORT BLAIR,KENDRIYA VIDYALAYA NO. 1,110101


In [ ]:
# # Group by relevant columns
# group_columns = ['state']
# agg_dict = {
#     'center_name':'first',
#     'center_number': 'first',
#     'city': 'first',
#     'marks': 'first'
# }
# df = df.groupby(group_columns).agg(agg_dict).reset_index()


In [ ]:
# %%time
# df = df.sort_values(by='marks', ascending=False).groupby('center_name').apply(lambda x: x.head(int(len(x)))).reset_index(drop=True)


In [ ]:
# df.to_csv('NEET_2024_RESULTS_processed.csv', index=False)

In [4]:
%%time
sentences = [word_tokenize(sent) for sent in df['state'].tolist()]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=multiprocessing.cpu_count())
model.save("word2vec.model")


CPU times: total: 1min 58s
Wall time: 1min 58s


In [5]:
from gensim.models import Word2Vec
model = Word2Vec.load("word2vec.model")

In [6]:
def get_vector(s):
    tokenized_phrase = word_tokenize(s)
    filtered_tokenized_phrase = [word for word in tokenized_phrase if word in model.wv.key_to_index]

     # Check if the filtered list is empty
    if len(filtered_tokenized_phrase) == 0:
        # Return a zero vector to avoid ZeroDivisionError
        return np.zeros(model.vector_size)
    
    # Compute the average vector
    
    average_vector = sum([model.wv.get_vector(word) for word in filtered_tokenized_phrase]) / len(filtered_tokenized_phrase)
    return average_vector

In [7]:
l1=df['state'].tolist()

In [8]:
%%time
import tqdm as tqdm
from annoy import AnnoyIndex
dim = 100
t = AnnoyIndex(dim,'euclidean')

for j,i in tqdm.tqdm(enumerate(l1),total=len(l1)):
    vector = get_vector(i)
    t.add_item(j,vector)

100%|█████████████████████████████████████████████████████████████████████| 2333120/2333120 [02:33<00:00, 15222.71it/s]

CPU times: total: 2min 30s
Wall time: 2min 33s


In [9]:
%%time
t.build(10)

CPU times: total: 1min 36s
Wall time: 20.6 s


True

In [10]:
index = list(range(len(l1)))
desc=df['center_number'].tolist()
desc_map = dict(zip(index,desc))
stem_map = dict(zip(index,l1))
hsn=df['center_name'].tolist()
hsn_map = dict(zip(index,hsn))

In [11]:
%%time
import time
start = time.time()
t.save('AnnIndex.ann')
end = time.time()
print('saving Cr9.ann completed in {}'.format(end-start))

saving Cr9.ann completed in 1.2604093551635742
CPU times: total: 703 ms
Wall time: 1.26 s


In [12]:
dim = 100
idx = AnnoyIndex(dim, 'euclidean')
idx.load('AnnIndex.ann')
model = Word2Vec.load("word2vec.model")

In [13]:
index = list(range(len(df)))
desc_map = dict(zip(index, df['center_number'].tolist()))
stem_map = dict(zip(index, df['state'].tolist()))
hsn_map = dict(zip(index, df['center_name'].tolist()))
count_map = dict(zip(index, df['marks'].tolist()))

In [14]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Ensure stopwords are downloaded
nltk.download('stopwords')

# Initialize the stopwords and stemmer
stop_words = set(stopwords.words('english'))
s_stemmer = SnowballStemmer("english")  # You can specify the language

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aryan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
def get_vector(s):
    tokenized_phrase = word_tokenize(s)
    filtered_tokenized_phrase = [word for word in tokenized_phrase if word in model.wv.key_to_index]
     # Check if the filtered list is empty
    if len(filtered_tokenized_phrase) == 0:
        # Return a zero vector to avoid ZeroDivisionError
        return np.zeros(model.vector_size)
    
    # Compute the average vector
    average_vector = sum([model.wv.get_vector(word) for word in filtered_tokenized_phrase]) / len(
        filtered_tokenized_phrase)
    return average_vector

def clean_query(query):
    x=re.sub(r'[^\w\s\d]', ' ', query)
    x=re.sub(r'(?<=\d)(?=\D)|(?<=\D)(?=\d)', ' ', x)
    x=re.sub(r'\b\w{1}\b', '', x)
    x=re.sub(r'(\_+)', '', x)
    x=re.sub(r'\s+', ' ', x)
    x=x.strip().lower()
    x=' '.join([i for i in x.split() if i not in stopwords])
    x=' '.join([s_stemmer.stem(i) for i in x.split()])
    return x
    
def search_similar_products(query, top_k=10):
    similar_indices = idx.get_nns_by_vector(get_vector(query), top_k)
    d={}
    results = []
    for i in similar_indices:
        results.append({
            'desc': desc_map[i],
            'stem': stem_map[i],
            'hsn': hsn_map[i],
            'count': count_map[i]
        })
    results=sorted(results, key=lambda x: x['count'], reverse=True)  
    l=['@'+str(i['hsn'])[:4] for i in results][:4]
    
    # for i,row in df_std[df_std['hsn'].isin(l)].iterrows(): 
    #     results.append({
    #         'std_hsn': row['hsn'],
    #         'std_hsn_desc': row['hsn_desc'],
    #     })        
    
    for i in results:
        if 'hsn' in i.keys():
            i['hsn']=str(i['hsn'])
            if len(i['hsn'])==3:
                i['hsn']='0'+i['hsn']
        # if 'std_hsn' in i.keys():
        #         i['std_hsn']=i['std_hsn'][1:]
    
    return results

In [16]:
while True:
    query = input("Enter your query: ")
    if query.lower() == "exit":
        break
    results = search_similar_products(query)
    print("Search Results:")
    for result in results:
        if 'desc' in result:
            print(f"center number: {result['desc']}")
            print(f"Center name: {result['hsn']}")
            print(f"Count: {result['count']}")
        elif 'std_hsn' in result:
            print(f"Standard HSN Code: {result['std_hsn']}")
            print(f"Description: {result['std_hsn_desc']}")
        print("-" * 30)

Enter your query:  Anda


Search Results:
center number: 380101
Center name: D.A.V. COLLEGE AMRITSAR (ID: C-27938)
Count: 564
------------------------------
center number: 380102
Center name: ARMY PUBLIC SCHOOL AMRITSAR
Count: 563
------------------------------
center number: 380103
Center name: DELHI PUBLIC SCHOOL G T ROAD AMRITSAR PUNJAB
Count: 264
------------------------------
center number: 380102
Center name: ARMY PUBLIC SCHOOL AMRITSAR
Count: 208
------------------------------
center number: 380103
Center name: DELHI PUBLIC SCHOOL G T ROAD AMRITSAR PUNJAB
Count: 204
------------------------------
center number: 380101
Center name: D.A.V. COLLEGE AMRITSAR (ID: C-27938)
Count: 191
------------------------------
center number: 380101
Center name: D.A.V. COLLEGE AMRITSAR (ID: C-27938)
Count: 178
------------------------------
center number: 380102
Center name: ARMY PUBLIC SCHOOL AMRITSAR
Count: 141
------------------------------
center number: 380102
Center name: ARMY PUBLIC SCHOOL AMRITSAR
Count: 64
-------

Enter your query:  Delhi


Search Results:
center number: 380101
Center name: D.A.V. COLLEGE AMRITSAR (ID: C-27938)
Count: 564
------------------------------
center number: 380102
Center name: ARMY PUBLIC SCHOOL AMRITSAR
Count: 563
------------------------------
center number: 380103
Center name: DELHI PUBLIC SCHOOL G T ROAD AMRITSAR PUNJAB
Count: 264
------------------------------
center number: 380102
Center name: ARMY PUBLIC SCHOOL AMRITSAR
Count: 208
------------------------------
center number: 380103
Center name: DELHI PUBLIC SCHOOL G T ROAD AMRITSAR PUNJAB
Count: 204
------------------------------
center number: 380101
Center name: D.A.V. COLLEGE AMRITSAR (ID: C-27938)
Count: 191
------------------------------
center number: 380101
Center name: D.A.V. COLLEGE AMRITSAR (ID: C-27938)
Count: 178
------------------------------
center number: 380102
Center name: ARMY PUBLIC SCHOOL AMRITSAR
Count: 141
------------------------------
center number: 380102
Center name: ARMY PUBLIC SCHOOL AMRITSAR
Count: 64
-------

KeyboardInterrupt: Interrupted by user